In [1]:
from brt.router._random import FusedRandomGatherRouter, FusedRandomScatterRouter
import brt.nn as nn
import brt
import torch


class FusedMoE(nn.Module):
    def __init__(self, expert_num):
        super().__init__()
        self.scatter_router = FusedRandomScatterRouter(
            route_num=expert_num, supported_capacities=[2, 4, 8, 16, 32, 64]
        )
        self.gather_router = FusedRandomGatherRouter(route_num=expert_num)

    def forward(self, inputs):
        route_results, reverse_indices, expert_capacities = self.scatter_router(inputs)
        route_results = self.gather_router(route_results, reverse_indices)
        return route_results


fused_moe = FusedMoE(expert_num=64)
input_tensor = torch.rand((64, 64))
output_tensor = fused_moe(input_tensor)
print(torch.isclose(output_tensor, input_tensor))


tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])


In [ ]:
from brt.router import RandomGatherRouter, RandomScatterRouter
import brt.nn as nn
import brt
import torch


class MoE(nn.Module):
    def __init__(self, expert_num):
        super().__init__()
        self.scatter_router = RandomScatterRouter(route_num=expert_num)
        self.gather_router = RandomGatherRouter(route_num=expert_num)

    def forward(self, inputs):
        route_results, reverse_indices = self.scatter_router(inputs)
        route_results = self.gather_router(route_results, reverse_indices)
        return route_results
